In [1]:
import csv
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point
import numpy as np
import eqcat as eqc
import scipy.spatial.distance
import time
from matplotlib.path import Path
import pyslabgrid.reckoner as reck

deg2km = 111.19492664455873
km2deg = 0.008993216059187306

# DO NOT RUN --- UNLESS YOU NEED TO. 

In [2]:
def is_within_interfacezone(evlon, evlat, szone, pbounds=None):
    if pbounds = None:
        finterp_file = {'hik': '../1SubductionModel/finterp/hik_finterp.npy',
                    'puy': '../1SubductionModel/finterp/puy_finterp.npy'}
        finterp = np.load(finterp_file[szone], allow_pickle=True)[()]
        pbounds = finterp['xbounds']
    if pbounds.contains(Point(evlon, evlat)):
        return True
    return False

# extract events on a profile/box
def get_box(cx,cy, strikeAn):
    # 
    if pbounds = None:
        finterp_file = {'hik': '../1SubductionModel/finterp/hik_finterp.npy',
                    'puy': '../1SubductionModel/finterp/puy_finterp.npy'}
        finterp = np.load(finterp_file[szone], allow_pickle=True)[()]
        finterp_strike = finterp['strikeAn']
        strikeAn = finterp_strike(cx,cy)

    # def reckon(x,y, d, bearing):
    # d is distance in km
    # x, y are corordinates in degrees
    # bearing or azimiuth in degrees        
    
    xm,ym = reck.reckon(cx,cy, 400, strikeAn-90)
    
    x1,y1 = reck.reckon(xm,ym, 100, strikeAn-180)
    x2,y2 = reck.reckon(x1,y1, 200, strikeAn)
    x3,y3 = reck.reckon(x2,y2, 100, strikeAn+90)
    x4,y4 = reck.reckon(x3,y3, 100, strikeAn+180)
    
    bx = [x1,x2,x3,x4,x1]
    by = [y1,y2,y3,y4,y1]
    return bx, by
#
def get_inpolygon(X,Y, polygon):
    xs,ys = polygon
    tupVerts = []
    for x,y in zip(xs,ys):
        tupVerts.append((x,y))
    p = Path(tupVerts) # make a polygon

    points = np.vstack((X,Y)).T 
    IN = p.contains_points(points)
    return IN

In [5]:
def writeout_projmslab(lon, lat, dep, mag, year, szone, fout):
    #
    # get the earthquake catalogue for hikurangi
    qlat, qlon, qdep, qmag, qyear = [],[],[], [], []
    for x,y,z, m, yr in zip(lon, lat,dep, mag, year):
        if m<3.0:
            continue
        #if yr<1900:
        #    continue
        if z>300:
            continue
        if is_within_interfacezone(x, y, szone):
            qlon.append(x)
            qlat.append(y)
            qdep.append(z)
            qmag.append(m)
            qyear.append(yr)
    
    #
    if szone =='hik':
        ifile_highres = 'hikinterfacegrid_highres.csv'
    else:
        ifile_highres = 'puyinterfacegrid_highres.csv'

    interf_rows = []
    with open(ifile_highres, mode='r') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        next(csv_reader)
        next(csv_reader)
        for row in csv_reader:
            dep_deg = float(row[2])*km2deg
            interf_rows.append([float(row[0]), float(row[1]), dep_deg])
    interf_rows = np.array(interf_rows)
    
    for x, y, z, m, yr in zip(qlon, qlat, qdep, qmag, qyear):
        # Distance between all pairs of points
        d = scipy.spatial.distance.cdist(interf_rows, np.transpose([[x],[y],[z*km2deg]]))
        dmin = min(d)[0]
        mindx = np.where(d==dmin)[0]
        # slab
        slon, slat = interf_rows[mindx,0][0], interf_rows[mindx,1][0]
        sdep = interf_rows[mindx,2][0]*deg2km
        fout.write('\n%.4f, %.4f, %.4f, %.4f, %d, %.4f, %.4f, %.4f,' \
               %(x,y, z, m, yr, slon, slat, sdep))
        
        if z < sdep:
            fout.write('%.4f'%(-dmin*deg2km))
        else:
            fout.write('%.4f'%(dmin*deg2km))
        

In [4]:
catalogue = 'NZeqcat_Rollins13042020-subd-slab.csv'
catalogue_folder = '../2EventClassifcation/subduction_catalog/'

out_catalogue ='NZeqcat_Rollins13042020-subd-slab-proj2interface.csv'
out_folder = 'projected_catalogs/'

ecat = eqc.read_slabcatalogue(catalogue_folder+catalogue, onlyslab=True)
lon, lat, mag, dep = ecat['lon'], ecat['lat'], ecat['mag'], ecat['dep']
year =  ecat['year']

fout= open(out_folder+out_catalogue,'w')
fout.write('lon,lat, dep, mag, year, slat, slon, sdep, dproj')
szone = 'puy'
t = time.time()
writeout_projmslab(lon, lat, dep, mag, year, szone, fout)
szone = 'hik'
writeout_projmslab(lon, lat, dep, mag, year, szone, fout)
elapsed = time.time() - t
#print('time taken:', elapsed)
fout.close()